In [1]:
import numpy as np
import random
import simpy

In [15]:
μ = 0.92 # the capacity of each of n equal servers. 
ρ = 0.96  #represents the system load. In a single server system, it will be: ρ=λ/μ, ρ=λ/(nμ) 
n_servers = np.array([1, 2, 4]) 
λ = (np.multiply(μ*ρ, n_servers)) #the arrival rate into the system as a whole.
##print(λ)

service_time_dist = ["M", "H", "D"]

SPTF = False # shortest processing time first 

arrival_time = []
service_time = []
waiting_time = []

SIM_TIME = 10_000

RANDOM_SEED = 142

RUNS = 1000


class Queue(object):
    """blah

    """
    def __init__(self, env, arrival_rate, num_servers, server_capacity, service_time_dist, SPTF):
        
        self.env = env
        self.arrival_rate = arrival_rate
        self.servers = simpy.Resource(env, num_servers)
        self.mu = server_capacity
        self.service_time_dist = service_time_dist #distribution
        self.SPTF = SPTF
        self.server_sjf = simpy.resources.resource.PriorityResource(env, 1) # only in case of num_servers = 1 
     
    
    # Functions for service time distribution

    def hyperexponential():
        """blah

        """
        rando = random.randint(0, 3)
        if rando == 0:
            t = random.expovariate(1.0 / 5.0) # average service time???
        else:
            t = random.expovariate(1.0 / 1.0)
        return t
    
        
    def server(self, customer):
        """blah

        """
        
        if self.service_time_dist == "M": 
            t = random.expovariate(1.0 / self.mu) # markovian
        
        elif self.service_time_dist == "H": 
            t = hyperexponential()
        
        else:
            t = self.mu # deterministic
        
        service_time.append(t)    
        yield self.env.timeout(t)
        #print("Service accomplished at %.2f." % (env.now))
        
        
        
def customer(env, name, q):
    """blah

    """
    
    arrive = env.now
    arrival_time.append(arrive)
    
    with q.servers.request() as request:
        yield request

        service_start = env.now
        waiting_time.append(service_start - arrive)
        #print('%s starts receiving the service at %.2f.' % (name, service_start))
        yield env.process(q.server(name))
        
        #print('%s leaves the service place at %.2f.' % (name, env.now))
        
    #print("Arrival time", arrival_time)
    #print("Service time", service_time)
    #print("Waiting time", waiting_time)

        
        
def sim_setup(env, arrival_rate, num_servers, server_capacity, service_time_dist, SPTF):
    """blah

    """

    # Create the queue
    queue = Queue(env, arrival_rate, num_servers, server_capacity, service_time_dist, SPTF)
    
    c = 0 # c for customer

    #generate new customers
    while True:
        arrive_t = random.expovariate(1.0/arrival_rate)
        yield env.timeout(arrive_t)
        arrival_time.append(arrive_t)
        c += 1
        #print('%.1f Arrival of customer  %d' % (env.now, c))
        env.process(customer(env, 'Customer %d' % c, queue))
    
        

# Setup and start the simulation
#print('Service')
random.seed(RANDOM_SEED)
print(n_servers)
print(λ[0])

# Create an environment and start the setup process
env = simpy.Environment()
env.process(sim_setup(env = env, arrival_rate = λ[0], num_servers = n_servers[0], server_capacity = μ, service_time_dist = "D", SPTF = False))
# Execute!
env.run(until=SIM_TIME)


[1 2 4]
0.8832


In [13]:
print(waiting_time)

[0.0, 0.0, 0.3303892821847998, 0.6625333160384623, 1.208388999143116, 0.5440462144827194, 1.1958424301220738, 2.0247951935539232, 2.778368074327311, 2.9782579561665763, 2.1068932471057185, 2.346544194065693, 3.2314674371157253, 3.897732269063111, 4.725145506655018, 5.311239453118308, 5.878859711785582, 4.708531861472451, 5.044736002964157, 4.456652883868674, 3.9329655387144307, 4.740311725762824, 5.532747605459292, 3.9385933106427657, 4.854386643683366, 5.260450163182771, 4.772078753682283, 5.447469392760759, 5.10811113232802, 4.1334116188955825, 3.4276867143238903, 4.039130948550042, 4.683311695233307, 5.224155330482642, 6.116548061709551, 7.017018924442237, 7.699314704437718, 8.43770356812568, 8.176619211945955, 8.736463417117559, 9.60631887315839, 10.207438005200672, 11.009374243960739, 11.909242958815934, 12.019866921721036, 12.696357274929738, 13.341043411244833, 14.072336459537311, 14.1656882501313, 13.697922370340642, 12.161541855750649, 10.837637561769085, 11.52097721339409, 11

In [17]:
print(np.mean(waiting_time) + np.mean(service_time))
print(1/(μ - λ[0]) )

95.2988746481029
27.17391304347822
